# Chapter 5

<a target="_blank" href="https://colab.research.google.com/github/wandb/edu/blob/main/rag-advanced/notebooks/Chapter05.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<!--- @wandbcode{rag-course-05} -->

## Retrieval and Re-ranking

In this chapter, we will focus on improving the retrieval quality and employ reranking to select the best context for our LLM.

In the last chapter, we added Query Enhancement module which was able to derive sub-queries from the user query. We retrieved contexts for these sub-queries. We did a basic deduplication of the total context to reduce the same context. However on a semantic level there might be few more contexts that we can drop thus reducing the input token cost.

When the same context is retrieved a few times, we can use this knowledge to fuse the confidence scores of such contexts and ask our LLM to focus more on them. Once can employ many strategies to improve the quality of contexts we provide our LLM.

Let's see a few here.

In [1]:
# @title Setup
!git clone https://github.com/wandb/edu.git
%cd edu/rag-advanced
!pip install -qqq -r requirements.txt
%cd notebooks

import nltk

nltk.download("wordnet")
nltk.download("punkt")

fatal: destination path 'edu' already exists and is not an empty directory.
/Users/michaelliu/workspace/wandb-edu/wandb-edu/rag-advanced/notebooks/edu/rag-advanced
/Users/michaelliu/workspace/wandb-edu/wandb-edu/rag-advanced/notebooks/edu/rag-advanced/notebooks


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/michaelliu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michaelliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import asyncio

import nest_asyncio

nest_asyncio.apply()

import weave

In [4]:
WANDB_PROJECT = "rag-course"

weave_client = weave.init(WANDB_PROJECT)

Logged in as Weights & Biases user: michaelyliu6.
View Weave data at https://wandb.ai/michaelyliu6-none/rag-course/weave


We will download the chunked data from chapter 3

In [5]:
# Reload the data from Chapter 3
chunked_data = weave.ref(
    "weave:///rag-course/rag-course/object/chunked_data:Ij9KThmiZQ9ljpCm8rVXTJlCaAbY2qC0zX6UJkBWHQ0"
).get()
# uncomment the next line to get the chunked data from weave from your own project instead
# chunked_data = weave.ref("chunked_data:latest").get()

chunked_data.rows[:2]
chunked_data = list(map(dict, chunked_data.rows[:]))

## Embedding based retriever

In the past chapters we were using TF-IDF and BM25 based retrievers. The vector representation from these methods are not "dense" i.e, they were not trained on billions or trillions of tokens. LLMs today embed the tokens into a dense representation. These embeddings are learned during the training process and are used to represent the tokens in the high-dimensional vector space. They have a denser knowledge of different lingustic patterns.

One stratightforward way of improving our retriever is to use a `DenseRetriever` which uses the embedding model (Cohere embedding model here) to embed the chunks and uses the same embedding model to embed the query.

## Reranking the contexts

With more contexts it is important to pick the one that adds more knowledge about the given query. For this, a re-ranking model is used that calculates a matching score for a given query and document pair. This score can then be utilized to rearrange vector search results, ensuring that the most relevant results are prioritized at the top of the list. Cohere comes with it's own re-ranking model and is quite popular.


In [6]:
from scripts.reranker import CohereReranker
from scripts.retriever import DenseRetriever, DenseRetrieverWithReranker
from scripts.utils import display_source

display_source(DenseRetriever)
display_source(CohereReranker)
display_source(DenseRetrieverWithReranker)

   1 class DenseRetriever(weave.Model):                                                                            
   2     """                                                                                                       
   3     A retriever model that uses dense embeddings for indexing and searching documents.                        
   4                                                                                                               
   5     Attributes:                                                                                               
   6         vectorizer (Callable): The function used to generate embeddings.                                      
   7         index (np.ndarray): The indexed embeddings.                                                           
   8         data (list): The data to be indexed.                                                                  
   9     """                                                                                                       
  10                                                                                                               
  11     vectorizer: Callable = sync_embed                                                                         
  12     index: np.ndarray = None                                                                                  
  13     data: list = None                                                                                         
  14                                                                                                               
  15     def index_data(self, data):                                                                               
  16         """                                                                                                   
  17         Indexes the provided data using dense embeddings.                                                     
  18                                                                                                               
  19         Args:                                                                                                 
  20             data (list): A list of documents to be indexed. Each document should be a dictionary              
  21                          containing a key 'cleaned_content' with the text to be indexed.                      
  22         """                                                                                                   
  23         self.data = data                                                                                      
  24         docs = [doc["cleaned_content"] for doc in data]                                                       
  25         embeddings = self.vectorizer(docs)                                                                    
  26         self.index = np.array(embeddings)                                                                     
  27                                                                                                               
  28     @weave.op()                                                                                               
  29     def search(self, query, k=5):                                                                             
  30         """                                                                                                   
  31         Searches the indexed data for the given query using cosine similarity.                                
  32                                                                                                               
  33         Args:                                                                                                 
  34             query (str): The search query.                                                                    
  35             k (int): The number of top results to r

   1 class CohereReranker(weave.Model):                                                                            
   2     """                                                                                                       
   3     A class to rerank documents using Cohere's reranking model.                                               
   4     """                                                                                                       
   5                                                                                                               
   6     model: str = "rerank-english-v3.0"                                                                        
   7                                                                                                               
   8     @weave.op()                                                                                               
   9     def rerank(self, query, docs, top_n=None):                                                                
  10         """                                                                                                   
  11         Reranks the given documents based on their relevance to the query.                                    
  12                                                                                                               
  13         Args:                                                                                                 
  14             query (str): The query string.                                                                    
  15             docs (List[Dict[str, Any]]): A list of documents to be reranked.                                  
  16             top_n (int, optional): The number of top documents to return. Defaults to None.                   
  17                                                                                                               
  18         Returns:                                                                                              
  19             List[Dict[str, Any]]: A list of reranked documents with relevance scores.                         
  20         """                                                                                                   
  21         client = cohere.Client(os.environ["COHERE_API_KEY"])                                                  
  22         documents = [doc["text"] for doc in docs]                                                             
  23         response = client.rerank(                                                                             
  24             model=self.model, query=query, documents=documents, top_n=top_n or len(docs)                      
  25         )                                                                                                     
  26                                                                                                               
  27         outputs = []                                                                                          
  28         for doc in response.results:                                                                          
  29             reranked_doc = docs[doc.index]                                                                    
  30             reranked_doc["relevance_score"] = doc.relevance_score                                             
  31             outputs.append(reranked_doc)                                                                      
  32         return outputs[:top_n]                                                                                
  33                                                                                                               
  34     @weave.op()                                                                                               
  35     def predict(self, query, docs, top_n=None):    

   1 class DenseRetrieverWithReranker(weave.Model):                                                                
   2     """                                                                                                       
   3     A retriever model that uses dense embeddings for retrieval and a reranker for re-ranking the results.     
   4                                                                                                               
   5     Attributes:                                                                                               
   6         retriever (DenseRetriever): The dense retriever model.                                                
   7         reranker (CohereReranker): The reranker model.                                                        
   8     """                                                                                                       
   9                                                                                                               
  10     retriever: DenseRetriever = DenseRetriever()                                                              
  11     reranker: CohereReranker = CohereReranker()                                                               
  12                                                                                                               
  13     def index_data(self, data):                                                                               
  14         """                                                                                                   
  15         Indexes the provided data using the dense retriever.                                                  
  16                                                                                                               
  17         Args:                                                                                                 
  18             data (list): A list of documents to be indexed. Each document should be a dictionary              
  19                          containing a key 'cleaned_content' with the text to be indexed.                      
  20         """                                                                                                   
  21         self.retriever.index_data(data)                                                                       
  22                                                                                                               
  23     @weave.op()                                                                                               
  24     def predict(self, query: str, top_k: int = None, top_n: int = None):                                      
  25         """                                                                                                   
  26         Predicts the top-n results for the given query after re-ranking.                                      
  27                                                                                                               
  28         Args:                                                                                                 
  29             query (str): The search query.                                                                    
  30             top_k (int, optional): The number of top results to retrieve before re-ranking. Default is None.  
  31             top_n (int, optional): The number of top results to return after re-ranking. Default is None.     
  32                                                                                                               
  33         Returns:                                                                                              
  34             list: A list of dictionaries containing the source, text, and score of the top-n results.         
  35         """                                        

Let's initialize the `DenseRetriever` and index the data.

In [7]:
dense_retriever = DenseRetriever()
dense_retriever.index_data(chunked_data)

In [8]:
from scripts.retrieval_metrics import IR_METRICS

eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

retrieval_evaluation = weave.Evaluation(
    name="Dense Retrieval Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k": 10},
)

dense_retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(dense_retriever))

Evaluated 1 of 20 examples

Evaluated 2 of 20 examples

Evaluated 3 of 20 examples

Evaluated 4 of 20 examples

Evaluated 5 of 20 examples

...

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'compute_hit_rate': {'mean': 0.33},
    'compute_mrr': {'mean': 0.13351587301587303},
    'compute_ndcg': {'mean': 0.3681740932338108},
    'compute_map': {'mean': 0.6630419973544973},
    'compute_precision': {'mean': 0.28484126984126984},
    'compute_recall': {'mean': 0.5996428571428571},
    'compute_f1_score': {'mean': 0.37505827505827505},
    'model_latency': {'mean': 0.5061103701591492}
}

🍩 https://wandb.ai/michaelyliu6-none/rag-course/r/call/01947292-322a-78a0-9469-57c12e692b4d


Let's initialize the `DenseRetrieverWithReranker` and index the data.

In [9]:
dense_retriever_rerank = DenseRetrieverWithReranker()
dense_retriever_rerank.index_data(chunked_data)

In [10]:
from scripts.retrieval_metrics import IR_METRICS

eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

retrieval_evaluation = weave.Evaluation(
    name="Dense Retrieval Rerank Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "top_k": 20, "top_n": 10},
)

dense_retrieval_scores = asyncio.run(
    retrieval_evaluation.evaluate(dense_retriever_rerank)
)

Evaluated 1 of 20 examples

Evaluated 2 of 20 examples

Evaluated 3 of 20 examples

Evaluated 4 of 20 examples

Evaluated 5 of 20 examples

...

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'compute_hit_rate': {'mean': 0.49000000000000005},
    'compute_mrr': {'mean': 0.24516666666666667},
    'compute_ndcg': {'mean': 0.30453573133804634},
    'compute_map': {'mean': 0.5564583333333333},
    'compute_precision': {'mean': 0.4741666666666666},
    'compute_recall': {'mean': 0.5083333333333333},
    'compute_f1_score': {'mean': 0.47465367965367966},
    'model_latency': {'mean': 2.368951070308685}
}

🍩 https://wandb.ai/michaelyliu6-none/rag-course/r/call/01947292-3f1b-7e63-a7cc-8864b7e6316a


## Hybrid Retriever

Even though BM25 is an old model used for retrieval tasks, it is still the state-of-the-art on various benchmark. In machine learning, we ensemble a few weak classifiers to build a stronger classifier, we can adopt the same idea to our retriever pipeline.

Below we show the concept of hybrid retriever which uses two or more retrievers and retrievr chunks from all of them followed by re-ranking.

### Fusion reranking

Since the indiviual retrievers are gonna retrieve the same chunks most of the time, simple re-ranking will not be beneficial since the same chunk from each retriever will have similar score, thus the top_k after re-ranking will have more or less the same context.

Instead, we iterate through all the chunks and fuse the score of similar chunk. Finally we sort on the basis of this fused score and return top_k chunks.

In [11]:
from scripts.retriever import HybridRetrieverReranker

display_source(HybridRetrieverReranker)

hybrid_retriever = HybridRetrieverReranker()

   1 class HybridRetrieverReranker(weave.Model):                                                                   
   2     """                                                                                                       
   3     A hybrid retriever model that combines sparse and dense retrieval methods and uses a reranker for final ra
   4                                                                                                               
   5     Attributes:                                                                                               
   6         sparse_retriever (BM25Retriever): The sparse retriever model using BM25.                              
   7         dense_retriever (DenseRetrieverWithReranker): The dense retriever model with a reranker.              
   8         fusion_ranker (FusionRanker): The fusion ranker to combine sparse and dense retrievals.               
   9         ranker (CohereReranker): The final reranker model.                                                    
  10     """                                                                                                       
  11                                                                                                               
  12     sparse_retriever: BM25Retriever = BM25Retriever()                                                         
  13     dense_retriever: DenseRetrieverWithReranker = DenseRetrieverWithReranker()                                
  14     fusion_ranker: FusionRanker = FusionRanker()                                                              
  15     ranker: CohereReranker = CohereReranker()                                                                 
  16                                                                                                               
  17     def index_data(self, data):                                                                               
  18         """                                                                                                   
  19         Indexes the provided data using both sparse and dense retrievers.                                     
  20                                                                                                               
  21         Args:                                                                                                 
  22             data (list): A list of documents to be indexed. Each document should be a dictionary              
  23                          containing a key 'cleaned_content' with the text to be indexed.                      
  24         """                                                                                                   
  25         self.sparse_retriever.index_data(data)                                                                
  26         self.dense_retriever.index_data(data)                                                                 
  27                                                                                                               
  28     @weave.op()                                                                                               
  29     def predict(self, query: str, top_k: int = None, top_n: int = None):                                      
  30         """                                                                                                   
  31         Predicts the top-n results for the given query after re-ranking.                                      
  32                                                                                                               
  33         Args:                                                                                                 
  34             query (str): The search query.                                                                    
  35             top_k (int, optional): The number of to

In [12]:
hybrid_retriever.index_data(chunked_data)

In [13]:
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

retrieval_evaluation = weave.Evaluation(
    name="Dense Retrieval Rerank Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "top_k": 20, "top_n": 10},
)

hybrid_retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(hybrid_retriever))

Evaluated 1 of 20 examples

Evaluated 2 of 20 examples

Evaluated 3 of 20 examples

Evaluated 4 of 20 examples

Evaluated 5 of 20 examples

...

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'compute_hit_rate': {'mean': 0.5900000000000001},
    'compute_mrr': {'mean': 0.26283333333333336},
    'compute_ndcg': {'mean': 0.3334039065800667},
    'compute_map': {'mean': 0.679498015873016},
    'compute_precision': {'mean': 0.5283333333333333},
    'compute_recall': {'mean': 0.445952380952381},
    'compute_f1_score': {'mean': 0.4589682539682539},
    'model_latency': {'mean': 5.778160703182221}
}

🍩 https://wandb.ai/michaelyliu6-none/rag-course/r/call/01947292-5c3c-70e0-9c23-a6b2f45c4f44
